In [18]:
import os

import tensorflow as tf
import keras 

import numpy as np

from PIL import Image

from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import concatenate

from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K

from tensorflow.keras.backend import set_session, clear_session, get_session

In [19]:
# Weighted categorical crossentropy
#
# source: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d
#

def weighted_cce(y_true, y_pred, weights=np.array([4,1,1])):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, -1)
    
    weights = K.variable(weights)
    
    return loss

In [20]:
# calculate the dice coefficient of a 
# predicted and true image tensors
#

def dice_coef(y_true, y_pred, smooth=1):
    
    # calculate the magnitude of the intersection between
    # the predicted and true tensors
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    
    # calculate the magnitude of the union of the 
    # predicted and true tensors
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

In [21]:
def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [22]:
def get_np_array_images_from_directory(dir_path, image_files=None):
    
    # container for image arrays
    np_arrays = []
    
    # if image_files wasn't specified read all images from directory
    if image_files is None:
        image_files = os.listdir(dir_path)
        image_files = sorted(image_files, key=lambda s:int(s.split('_')[2].split('.')[0]))
    
    # construct the appropriate absolute paths for each image
    image_paths = [os.path.join(dir_path, filename) for filename in image_files]
    
    # read, resize, and convert each image to a Numpy array
    for path in image_paths:
        
        # open image
        img = Image.open(path)
        img = img.convert("RGB")
        
        # resize image
        img = img.resize((256, 256))
        
#         # convert to binary image if is_mask is True
#         if is_mask:
#             img = img.convert("1")
        
        # convert image to Numpy array 
        array = np.array(img)
        
#         # ensure mask images have a single color channel dimension
#         if is_mask:
#             array = array.reshape(256, 256, 1)
        
        # add the image to the image arrays container
        np_arrays.append(array)
        
    return np.array(np_arrays)

In [23]:
# delete all imagese in the predictions 
# directory
#

def clear_old_predictions(predictions_dir):
    files = os.listdir(predictions_dir)
    for file in files:
        file_path = os.path.join(predictions_dir, file)
        os.remove(file_path)

In [24]:
# convert a Numpy array containing the information 
# for a mask image to a pillow image
#

def get_mask_from_binary_array(np_array) -> Image:
    
    # extract dimensions of image
    height = np_array.shape[0]
    width = np_array.shape[1]
    
    # cut off the channels dimension
    # which should be 1
    np_array = np_array.reshape(height, width)
    
    # convert to 255 black and white representation
    np_array *= 255
    
    return Image.fromarray(np_array)

In [25]:
# set gpu configuration for this system
#

config = tf.compat.v1.ConfigProto( device_count={'GPU': 1, 'CPU':8})
sess = tf.compat.v1.Session(config=config)
keras.backend.set_session(sess)

In [26]:
top_model_dir = "top_models"

model_filenames = os.listdir(top_model_dir)

model_filepaths = [os.path.join(top_model_dir, filename) for filename in model_filenames]

dependencies = {
    "dice_coef": dice_coef,
    "dice_loss": dice_loss,
    "weighted_cce": weighted_cce
}

testing_images_dir = "celeb_pics/"
predictions_dir = "predictions"

In [27]:
testing_image_arrays = get_np_array_images_from_directory(testing_images_dir)
testing_image_arrays = testing_image_arrays.astype("float32") / 255.0

In [28]:
print(model_filepaths)

['top_models\\fcn8_1.hdf5', 'top_models\\fcn8_2_789.hdf5', 'top_models\\fcn8_4_798.hdf5', 'top_models\\unet_1_790.hdf5', 'top_models\\unet_2_807.hdf5']


In [29]:
model = load_model(model_filepaths[4], custom_objects=dependencies)

In [30]:
predictions = model.predict(testing_image_arrays, batch_size=1)

In [31]:
# for filepath in model_filepaths[1:]:
#     model = load_model(filepath, custom_objects=dependencies)
#     predictions += model.predict(testing_image_arrays, batch_size=1)

In [32]:
classes = (predictions.argmax(axis=-1) == 0).astype("int")

In [34]:
# clear the predictions directory
#
clear_old_predictions(predictions_dir)

save_tag = "test_mask"

i = 1
for c in classes:
    img = get_mask_from_binary_array(c)
    img = img.resize((250, 250))
    img.save(os.path.join(predictions_dir, "{}_{}.png".format(save_tag, str(i))), format="PNG")
    i += 1